In [ ]:
import pyodbc
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math


In [ ]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

In [ ]:
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

In [ ]:
FD_path = Path( home, "desktop" , 'FD_all.xlsx')
FD = pd.read_excel( FD_path, sheet_name='Sheet1' )

In [ ]:
for index, row in FD.iterrows():
    ODM = row['ODM']
    Item = row['Item']
    Commodity = row['Commodity']
    FV = row['FV']
    Platform = row['Platform']
    HP_PN = row['HP_PN']
    Supplier = row['Supplier']
    PN = row['HP PN']
    ReportDate = row['ReportDate']
    FDdate = row['FDdate']
    FDQty = row['FDQty']
    BuyerName = row['BuyerName']

    cursor.execute(f"INSERT INTO CSI.GPS.GPS_tbl_ops_fd_temp ( ODM,Item,Commodity,FV,Platform,Supplier,[HP PN],FDdate,FDQty,Reportdate,BuyerName )\
                    VALUES('{ODM}','{Item}','{Commodity}','{FV}','{Platform}','{Supplier}','{PN}','{FDdate}','{FDQty}','{ReportDate}','{BuyerName}')")
conn.commit()
conn.close()